<img style="float: left;" src="https://www.apachecollege.org/wp-content/uploads/2018/10/cropped-SCAC-logo-2014-1.png" width="175" height="175"><img style="float: right;" src="https://pbs.twimg.com/profile_images/1537109064093532160/mG03dW9G_400x400.jpg" width="175" height="175">


Welcome to the ESIIL-San Carlos Apache College Python Training. My name Nate Quarderer and I'm the Education Director at the Environmental Data Science Innovation & Inclusion Lab (ESIIL). Today I'm going to demonstrate for you some GIS + Earth Data Science (EDS) applications using Python.

In **Part 1 (Working with Spatial Data in Python)** we'll show you how to create maps using data from the U.S. Census Bureau, specifically **The American Indian/Alaska Native/Native Hawaiian (AIANNH) Areas Shapefile:** https://catalog.data.gov/dataset/tiger-line-shapefile-2020-nation-u-s-american-indian-alaska-native-native-hawaiian-aiannh-areas

In **Part 2 (Working with Time-Series Data in Python)** you will learn how to access data from the [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) at **NOAA**. We will then use those data to create a time-series plot of temperature or precipitation over time.

In **Part 3 (Create Interactive Maps with Folium)** you will learn how to create an interactive map of the San Carlos Apache Reservation using folium.

# Part 1: Working with Spatial Vector Data in Python

### About Spatial Vector Data

Vector data are composed of discrete geometric locations (x, y values) known as vertices that define the “shape” of the spatial object. The organization of the vertices determines the type of vector that you are working with. There are three types of vector data:

**Points:** Each individual point is defined by a single x, y coordinate. Examples of point data include: sampling locations, the location of individual trees or the location of plots.

**Lines:** Lines are composed of many (at least 2) vertices, or points, that are connected. For instance, a road or a stream may be represented by a line. This line is composed of a series of segments, each “bend” in the road or stream represents a vertex that has defined x, y location.

**Polygons:** A polygon consists of 3 or more vertices that are connected and “closed”. Thus, the outlines of plot boundaries, lakes, oceans, and states or countries are often represented by polygons.


<img style="float: left;" src="https://www.earthdatascience.org/images/earth-analytics/spatial-data/points-lines-polygons-vector-data-types.png">


> ### ✨ Read more about working with spatial data using Python in our free, open EDS textbook, [here](https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-spatial-data/use-vector-data/). ✨


In [ ]:
# Import Python packages
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium

In [ ]:
# Land areas url
aiannh_url = "https://www2.census.gov/geo/tiger/TIGER2020/AIANNH/tl_2020_us_aiannh.zip"

# Open land area boundaries
aiannh_boundary = gpd.read_file(aiannh_url)
print(aiannh_boundary)

# Plot land area boundaries
fig, ax = plt.subplots(figsize=(20,12))
aiannh_boundary.plot(color='gold',
                         edgecolor='purple',
                         ax=ax)
ax.set_title("Map of Federally Recognized American Indian \nReservations and Off-Reservation Trust Land Areas", fontsize=30)
plt.show()

In [ ]:
# Open US State boundary
us_url = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip"

us_gdf = gpd.read_file(us_url)
us_gdf

In [ ]:
# Select AZ
az_bndry = us_gdf.loc[us_gdf['NAME'] == 'Arizona']

print("The CRS of the AZ shapefile is:", az_bndry.crs)
az_bndry.plot(color='gold',
                    edgecolor='black')

In [ ]:
# Clip AIANNH boundary to AZ boundary
aiannh_az = aiannh_boundary.clip(az_bndry.geometry)
aiannh_az

In [ ]:
# Plot clipped AIANNH boundary
fig, ax = plt.subplots(figsize=(10,7))
az_bndry.plot(color='linen',
              edgecolor='dimgrey',
              ax=ax)


aiannh_az.plot(ax=ax,
               edgecolor='black',
               column='NAME',
               legend=True,
               alpha=0.8)

# Define and place legend
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.6, 1.0, 0., 0.))

plt.title("Federally Recognized American Indian Reservations \nand Off-Reservation Trust Land Areas (AZ)", fontsize=20)
plt.show()

In [ ]:
# Select each reservation and plot separately
navajo_nation_bndry = aiannh_boundary.loc[aiannh_boundary['NAME'] == 'Navajo Nation']
san_carlos_bndry = aiannh_boundary.loc[aiannh_boundary['NAME'] == 'San Carlos']

In [ ]:
# Plot Navajo Nation
navajo_nation_bndry.plot(color='beige',
                         edgecolor='black',
                         figsize=(20,12))

In [ ]:
# Plot San Carlos
san_carlos_bndry.plot(color='beige',
                         edgecolor='black',
                         figsize=(20,12))

# Part 2: Working with Time-Series Data in Python

Here we're using the NOAA National Centers for Environmental Information (NCEI) [Access Data Service](https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation) application progamming interface (API) to request data from their web servers. We will be using data collected as part of the [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) program at NOAA.

For this example we're requesting data near the **Black River** (station ID USC00020808) located on the San Carlos Reservation (**33.4783°, -109.7516°**).

https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USC00020808/detail


> ### ✨ Read more about working with time-series data using Python in our free, open EDS textbook, [here](https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/introduction-to-time-series-in-pandas-python/) ✨

In [ ]:
# Use NCEI API to pull data from the station @ the Black River, AZ (San Carlos Apache)
black_river_api_url = (
           "https://www.ncei.noaa.gov/access/services/data/v1"
           "?dataset=daily-summaries&"
           "dataTypes=TAVG,TMAX,TMIN,PRCP&"
           "stations=USC00020808&"
           "startDate=1948-07-01&"
           "endDate=2023-10-01&"
           "includeStationName=true&"
           #"includeStationLocation=1&"
           "units=standard"
           )

In [ ]:
# Open and clean data with pandas
black_river_df = pd.read_csv(black_river_api_url,
                              na_values=["-99"],
                              index_col=["DATE"],
                              parse_dates=["DATE"])
black_river_df

In [ ]:
# Plot data with matplotlib
fig, ax = plt.subplots(figsize=(10,6))

black_river_df.plot(y='PRCP',
                 ax=ax,
                 ylabel='Precipitation (in)',
                 xlabel='',
                 title='Precipitation (in) @ Black River, AZ\nSan Carlos Reservation\n1948-2023')

plt.show()

# Part 3: Creating Interactive Maps using Folium
Here you will create an interactive map of the Black River USGS gage location and the San Carlos Apache Reservation using folium.




In [ ]:
# Create interactive map using folium

# River gage info
gage_lat = 33.4783
gage_long = -109.7516
gage_location_name = 'River Gage - Black River'

# San Carlos Apache College info
scac_lat = 33.3522
scac_long = -110.4519
scac_loc_name = 'San Carlos Apache College'

# Create a Folium map object
m = folium.Map(location=[scac_lat,scac_long], zoom_start=15)

# Convert the GeoDataFrame to GeoJSON format
geojson_data = san_carlos_bndry.to_json()

# Add the GeoJSON data to the map as a GeoJson layer
folium.GeoJson(geojson_data).add_to(m)

# Add marker for river gage
folium.Marker(
    location=[gage_lat, gage_long],
    popup=gage_location_name,
    icon=folium.Icon(color='black')
).add_to(m)


# Add marker for San Carlos Apache College
folium.Marker(
    location=[scac_lat, scac_long],
    popup=scac_loc_name,
    icon=folium.Icon(color='purple')
).add_to(m)


# Save the map as an HTML file or display it inline
m.save('map.html')
# OR
m